In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import importlib.resources as pkg_resources
from pistachio import transfer_matrix as tm
from pistachio import default

## Build a structure, layer by layer

First, we will build a multi-layered optical structure by creating individual Layer classes and then adding them to a structure.

Let's build a simple structure with some material that can be modeled as a Lorentz oscillator sandwiched between two CaF2 plates with 20 μm spacing. First, we'll make our material using the complex [Lorentzian function](https://en.wikipedia.org/wiki/Cauchy_distribution) defined as

$$
L(\omega;\omega_0, f_0, \gamma) = \frac{f_0}{\omega^2 - \omega_0^2 + i\omega\gamma},
$$

where $f_0$ is the amplitude, $\omega_0$ is the center angular frequency, and $\gamma$ is phenomenological damping.

In [ ]:
def lorentzian(w, amp=1., w_0=0., gamma=1.):
	return amp / (w**2 - w_0**2 + 1j*w*gamma)

In [ ]:
# Set a center wavelength at 4 μm or 2500 cm-1
num_points = 1000
wavelength = np.linspace(3.5e-6, 4.5e-6, num_points)
center = 4.e-6
damping = 1.5e-8
amplitude = 1.e-14
lor = lorentzian(wavelength, amplitude, center, damping)

Define the complex refractive index as

$$
\displaystyle
n(\omega) = \sqrt{n_{b}^2 - L(\omega; \omega_0, f_0, \gamma)},
$$

where $n_b$ is the background refractive index and
$L(\omega; \omega_0, f_0, \gamma)$ is the complex Lorentzian function defined previously.

In [ ]:
n_eff = 1.5
refractive = np.sqrt(n_eff**2 - lor)

And let's plot this. Comment out either the real or imaginary trace to see the other trace better.

In [ ]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x=wavelength, y=refractive.real, name='real'))
fig.add_trace(go.Scatter(x=wavelength*10**6, y=refractive.imag, name='imaginary'))
fig.update_layout(title="Lorentz Oscillator Model",
                  xaxis_title="Wavelength (μm)",
                  yaxis_title="Refractive Index")

fig.show()

## Build a structure

Now that we have our middle layer, let's put together our structure, starting with the first CaF2 layer where 
the light will first enter. As long as refractive index data is in the `data/refractive_index` directory,
we can directly load it without the full path.

In [ ]:
# Initialize a new structure.
s = tm.Structure()

# If we print the structure, we see that nothing has been added yet.
s.print_structure()

In [ ]:
# make a CaF2 layer and add it to the structure
caf2 = tm.Layer(material='CaF2', thickness=0.)
caf2.get_data_from_csv('CaF2.csv')
s.add_layer(caf2)

# Now we have one layer
s.print_structure()

Now let's add our sample layer. We'll initialize it with a material name, the thickness we want, and a list
with a single arbitrary wavelength. This can't be empty, but it doesn't matter since we recalculate everything
according to the spectrum we want to produce.

In [ ]:
sample = tm.Layer(material='Sample', thickness=2.e-5, wavelengths=[1.e-6])

In [ ]:
sample.wavelengths = wavelength
sample.refractive_index = refractive.real
sample.extinction_coeff = refractive.imag
sample.set_complex_refractive(refractive.real, refractive.imag)
s.add_layer(sample)
s.print_structure()


We can also delete a layer. Add the sample again and print the structure configuration.
Then comment the `add_layer` function and uncomment the `delete_layer` function to try this.

In [ ]:
# s.add_layer(sample)
# s.delete_layer(3)
# s.print_structure()

We can also set layer parameters later.

In [ ]:
s.layers[1].thickness = 2.e-5

Finally, we add the last CaF2 layer the same as before.

In [ ]:
caf2 = tm.Layer(material='CaF2', thickness=0.)
caf2.get_data_from_csv('CaF2.csv')
s.add_layer(caf2)
s.print_structure()

Next we have to initialize the structure so that each layer has the same number of wavelengths.
Pistachio can do a few different calculations, including angle tuning, but we don't need that right now.
We set $\theta_i$ and $\theta_f$ equal to zero and the number of angles equal to 1, so that we only
calculate for zero-degree incidence angle. Then we want to sweep through wavelengths
3.5 μm - 4.5 μm. This is the same as the wavelengths set for our sample, but it could be a smaller
range within that. Be careful not to set a range outside the refractive index data
of any layer. Errors may occur, or the results may be unreliable.
Pistachio uses SciPy's interpolate method to generate new data. 

In [ ]:
theta = 0.0
polarization = 's-wave'
min_wl = 3.5e-6
max_wl = 4.5e-6
num_wavelengths = 5000
s.initialize_struct(theta, theta, 1, min_wl, max_wl, num_wavelengths, polarization)

Finally calculate transfer matrices for each layer at each wavelength and then the total transfer matrix
for the structure at each wavelength, `M_all`. Then calculate the transmittance and reflectance
for each transfer matrix, `T_all` and `R_all`.

In [ ]:
M_all = s.calculate_all_transfer_matrices(theta, polarization)
T_all, R_all = s.calculate_all_t_r(M_all)
T_percent = T_all * 100

In [ ]:
wavenumber = (1 / s.wavelengths) / 100

fig = go.Figure()
fig.add_trace(go.Scatter(x=wavenumber, y=T_percent))
fig.update_layout(title="Lorentzian oscillator material between two CaF2 plates", 
                  xaxis_title="Wavenumber (cm-1)",
                  yaxis_title="Transmittance (%)")
fig.show()

## Build a Fabry-Pérot microcavity from a pre-populated yaml config file

The `load_struct_from_config` method uses a yaml config file created by you to 
generate a multi-layered structure. In our example, there are two CaF2 layers and two
Au layers. You can see the config file itself in the `default` directory. 
The data for these layers are downloaded from refractiveindex.info, so
the number of data points between them differs. The middle layer is air, which just
has a single refractive index for all wavelengths that we consider here. 
The `load_struct_from_config` method will
refactor each of these layers so that they have the same set of wavelengths. Then
the transfer matrix can be calculated for each wavelength between the user-defined
max and min wavelengths.

Each layer is a class that you can access and modify.
A structure is essentially a list of layer classes with methods that can act on
all layers, such as performing layer transfer matrices and calculating the total transmittance
and reflectance for light propagating through the entire structure.

Here, we demonstrate calculating the transmittance and reflectance of light travelling 
through a Fabry-Pérot microcavity.

In [ ]:
yaml_file = 'fabry-perot.yaml'
with pkg_resources.path(default, yaml_file) as yml:
    yaml_config = os.path.abspath(yml)

fp = tm.Structure()
fp.load_struct_from_config(yaml_config)

In [ ]:
# Print the parameters of our structure.
fp.print_structure()

## Calculate Transfer Matrix, Transmittance, and Reflectance

Set the angle of incidence (zero degrees) and light polarization (s-wave). Then 
we calculate each layer transfer matrix at every wavelength and then generate the total
structure transfer matrix (again at each wavelength). A list of transfer matrices 
is returned. Then `calculate_all_t_r` calculates the transmittance and reflectance
from each provided total transfer matrix. From this we can plot the transmittance and 
reflectance spectrum for our structure.

In [ ]:
theta = 0.
polarization = 's-wave'
M_all = fp.calculate_all_transfer_matrices(theta, polarization)
T_all, R_all = fp.calculate_all_t_r(M_all)

In [ ]:
# Convert to wavenumbers (cm-1) and percent.

wavenumbers = (1 / fp.wavelengths) / 100
T_percent = T_all * 100
R_percent = R_all * 100

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=wavenumbers, y=T_percent, name='T'))
fig.add_trace(go.Scatter(x=wavenumbers, y=R_percent, name='R'))

fig.update_layout(title='Fabry-Pérot etalon of intra-mirror distance {} μm'.format(fp.layers[2].thickness*10**6),
                  xaxis_title='Wavenumber (cm-1)',
                  yaxis_title='% Transmittance/Reflectance')
fig.show()